In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

from IPython.display import HTML
from IPython.display import display

from openmm import *
from simtk.unit import *
import MDAnalysis as md
import nglview as ng
from sys import stdout

In [17]:
# These files are already preloaded to your ``data`` folder

pdb0_file = "data/villin_water.pdb"
pdb1_file = "data/polyALA.pdb"
pdb2_file = "data/polyGLY.pdb"
pdb3_file = "data/polyGV.pdb"

In [18]:
# 1. loading initial coordinates
pdb = PDBFile(pdb1_file)

In [19]:
# 2.  choosing a forcefield parameters
ff = ForceField("amber10.xml")
system = ff.createSystem(pdb.topology, nonbondedMethod=CutoffNonPeriodic)

In [21]:
# 3. Choose parameters of the experiment: temperature, pressure, box size, solvation, boundary conditions, etc...
temperature = 300*kelvin
frictionCoeff = 1/picosecond
time_step = 0.002*picoseconds
total_steps = 400*picoseconds / time_step

In [23]:
# 4. Choose an algorithm (integrator)
integrator = LangevinIntegrator(temperature, frictionCoeff, time_step)

In [24]:
# 5. Run simulation, saving coordinates time to time
## 5a. Create a simulation object
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

In [25]:
## 5b. Minimize energy
simulation.minimizeEnergy()

In [30]:
## 5c. Save coordinates to dcd file and energies to a standard output console
simulation.reporters.append(DCDReporter("data/polyALA_traj.dcd", 1000))
simulation.reporters.append(StateDataReporter(stdout, 5000, step=True, potentialEnergy=True,
                                              temperature=True, progress=True, totalSteps=total_steps))

In [31]:
## 5d. Run
simulation.step(total_steps)

102.5%,205000,4755.770090162754,278.0802779306465
#"Progress (%)","Step","Potential Energy (kJ/mole)","Temperature (K)"
102.5%,205000,4755.770090162754,278.0802779306465
105.0%,210000,4709.634234428406,282.43120932624356
105.0%,210000,4709.634234428406,282.43120932624356
107.5%,215000,4706.4244492948055,292.05316412261095
107.5%,215000,4706.4244492948055,292.05316412261095
110.0%,220000,4651.131392072886,310.87189244326083
110.0%,220000,4651.131392072886,310.87189244326083
112.5%,225000,4737.088874101639,296.0999097785986
112.5%,225000,4737.088874101639,296.0999097785986
115.0%,230000,4852.0043251514435,314.1053850304492
115.0%,230000,4852.0043251514435,314.1053850304492
117.5%,235000,4805.76481795311,299.3131168125324
117.5%,235000,4805.76481795311,299.3131168125324
120.0%,240000,4753.625875711441,312.32641815934505
120.0%,240000,4753.625875711441,312.32641815934505
122.5%,245000,4716.6938898563385,336.1210520975588
122.5%,245000,4716.6938898563385,336.1210520975588
125.0%,250000,4681

In [32]:
## Visualization
# Let's look at the trajectory
#6. Visualization
sys = md.Universe(pdb1_file, "data/polyALA_traj.dcd")
ng.show_mdanalysis(sys, gui=True)

/home/dbsejrgus226/miniconda3/envs/MD/lib/python3.12/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


NGLWidget(max_frame=199)

In [34]:
# End-to-end distance
# analysis of end-to-end distance
#choose terminal atoms

N_terminus = sys.select_atoms("resid 1 and name N")
C_terminus = sys.select_atoms("resid 25 and name C")

# go through the whole trajectory and compute distance between them for every frame
dist = []
for frame in sys.trajectory:
    dist.append(np.linalg.norm(N_terminus.positions - C_terminus.positions))

# the result is in the dist array
dist = np.array(dist)

In [49]:
# Number of hydrogen bonds
from MDAnalysis.analysis.hydrogenbonds import HydrogenBondAnalysis ## module for analysis of hydrogen bonds

## compute information about hbonds and write it to the 'hb.timeseries'
hb = HydrogenBondAnalysis(sys)
hb.run()

## print information for the first 10 frames
for frame in hb.timeseries[:10]:
    print(frame)

NoDataError: This Universe does not contain charge information